In [14]:
import requests
import json
from data_taking import BikeTimes

In [56]:
# request = requests.get('https://api.tfl.gov.uk/Place/Type/BikePoint')

In [57]:
# bikes9 = request.json()

In [24]:
# len(bikes)

780

In [58]:
# with open('./bikes_9.json', 'w+') as f:
#     f.write(json.dumps(bikes9, sort_keys=True, indent=4))

In [46]:
# f = open('bike_test_friday.json')

In [47]:
# bikes = json.load(f)

In [48]:
print(bikes[1]['commonName'])
for  i in bikes[1]['additionalProperties']:
    print(i['key'], i['value'])
print('lat', bikes[1]['lat'])
print('long', bikes[1]['lon'])

Phillimore Gardens, Kensington
TerminalName 001018
Installed true
Locked false
InstallDate 1278585780000
RemovalDate 
Temporary false
NbBikes 14
NbEmptyDocks 23
NbDocks 37
lat 51.499606
long -0.197574


In [55]:
5*(26//5)

25

In [16]:
from datetime import datetime

In [27]:
class BikeTimes():
    def __init__(self):
        self.date = datetime.now()
        self.hour = self.date.hour
        self.minute = self.date.minute
        self.url = 'https://api.tfl.gov.uk/Place/Type/BikePoint'
        self.data = requests.get(self.url).json()
        
    def save_data(self):
        with open(f'./data/bike_data_{self.hour}_{self.minute}.json', 'w+') as f:
            f.write(json.dumps(self.data, sort_keys = True, indent = 4))
        

In [28]:
BikeTimes().save_data()